In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
BATCH_SIZE = 64
TEST_BATCH_SIZE = 1000
EPOCHS = 14
LR = 0.001
LOG_INTERVAL = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
class MNIST_Polygon(datasets.MNIST):
    
    def __init__(self, path, train=True, download=False):
        super().__init__(root=path, train=train, download=download)
        
        data = self.data.view(-1, 784)
        data = list(data.unbind())
        data = [torch.arange(784)[(x > 0.8)] for x in data]
        lengths = [len(x) for x in data]
        max_length = max(lengths)
        data = [torch.Tensor(list(x) + (351 - len(x)) * [0]) for x in data]
        data = torch.stack(data).contiguous()
        self.data = torch.reshape(data, (len(data) , 1, 351))
        self.max_length = max_length
        
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        return x, y
        
    @property
    def raw_folder(self):
        return os.path.join(self.root, 'MNIST', 'raw')

    @property
    def processed_folder(self):
        return os.path.join(self.root, 'MNIST', 'processed')

In [4]:
dataset1 = MNIST_Polygon('~/Developer/datasets', train=True)
dataset2 = MNIST_Polygon('~/Developer/datasets', train=False)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=TEST_BATCH_SIZE)

In [54]:
class SimpleModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(351, 512, bias=False),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Linear(512, 256, bias=False), 
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            nn.Linear(256, 10)
        )
        
    def forward(self, x):
        return self.net(x[:, 0])

In [11]:
class SimpleModel2dFair(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(702, 324, bias=False),
            nn.BatchNorm1d(324),
            nn.ReLU(True),
            nn.Linear(324, 256, bias=False), 
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            nn.Linear(256, 10)
        )
        
    def forward(self, x):
        x = x[:, 0]
        x = torch.cat([x // 28, x % 28], -1)
        return self.net(x)

In [ ]:
class SimpleModel2d(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(702, 512, bias=False),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Linear(512, 256, bias=False), 
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            nn.Linear(256, 10)
        )
        
    def forward(self, x):
        x = x[:, 0]
        x = torch.cat([x // 28, x % 28], -1)
        return self.net(x)

In [30]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 32, 5),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Conv1d(32, 64, 5),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.25)
        )
        
        self.linear = nn.Sequential(
            nn.Linear(10944, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10)
        )


    def forward(self, x):
        x = self.conv(x)
        x = torch.flatten(x, 1)
        return self.linear(x)



In [75]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()), end="\r")


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = nn.CrossEntropyLoss(reduction='sum')
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:
model = SimpleModel2dFair().to(DEVICE)
summary(model, (1, 351))
optimizer = optim.Adam(model.parameters(), lr=LR)



Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 10]                  --
|    └─Linear: 2-1                       [-1, 324]                 227,448
|    └─BatchNorm1d: 2-2                  [-1, 324]                 648
|    └─ReLU: 2-3                         [-1, 324]                 --
|    └─Linear: 2-4                       [-1, 256]                 82,944
|    └─BatchNorm1d: 2-5                  [-1, 256]                 512
|    └─ReLU: 2-6                         [-1, 256]                 --
|    └─Linear: 2-7                       [-1, 10]                  2,570
Total params: 314,122
Trainable params: 314,122
Non-trainable params: 0
Total mult-adds (M): 0.63
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 1.20
Estimated Total Size (MB): 1.21


In [ ]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [59520/60000 (99%)]	Loss: 0.545012
Test set: Average loss: 1.3808, Accuracy: 5341/10000 (53.4%)

Train Epoch: 2 [59520/60000 (99%)]	Loss: 0.459200
Test set: Average loss: 1.0358, Accuracy: 6525/10000 (65.2%)

